RBF

In [62]:
from filtrar_datos import filtrar_datos_en_rango
from procesar_tableros import procesar_tableros
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np
import scipy
import scipy.io
from procesar_tableros import *
from filtrar_datos import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import xlsxwriter
class RBFNetwork:
    def __init__(self, num_centers, sigma, lambda_reg):
        self.num_centers = num_centers
        self.sigma = sigma
        self.lambda_reg = lambda_reg
        self.centers = None
        self.weights = None

    def _calculate_activations(self, X):
        #distances = scipy.spatial.distance.cdist(X, self.centers)
        distances = np.linalg.norm(X[:, None, :] - self.centers[None, :, :], axis=2)
        activations = np.exp(-distances**2 / (2 * self.sigma**2))
        return activations

    def fit(self, X, y):
        indices = np.random.choice(X.shape[0], self.num_centers, replace=False)
        self.centers = X[indices]
        activations = self._calculate_activations(X)
        
        # Añadir el término de regularización
        #A = activations.T @ activations
        #I = np.eye(A.shape[0])
        #self.weights = np.linalg.pinv(A + lambda_reg * I) @ activations.T @ y
        regularization_matrix = self.lambda_reg * np.eye(activations.shape[1])
        self.weights = np.linalg.solve(activations.T @ activations + regularization_matrix, activations.T @ y)


    def predict(self, X):
        activations = self._calculate_activations(X)
        return activations @ self.weights


Cargar datos

In [63]:
# Cargar datos
print("Cargando datos...")
file_path_1bit = "./10x10/10x10/1bit/DATASET_v7.mat"
file_path_2bit = "./10x10/10x10/2bit/DATASET_v7.mat"

mat_data1b = scipy.io.loadmat(file_path_1bit)
mat_data2b = scipy.io.loadmat(file_path_2bit)
print("Datos cargados.")

# Obtenemos los dataset
dataset1b = mat_data1b['DATASET']
dataset2b = mat_data2b['DATASET']

# Acceder a las variables dentro del archivo (1bit)
labels_1b = dataset1b['labels']
inputs_1b = dataset1b['input']
bits_1b = dataset1b['bits']
beams_1b = dataset1b['beams']

# Acceder a las variables dentro del archivo (2bit)
labels_2b = dataset2b['labels']
inputs_2b = dataset2b['input']
bits_2b = dataset2b['bits']
beams_2b = dataset2b['beams']

Cargando datos...
Datos cargados.


In [64]:

# Acceder a las variables dentro del archivo (1bit)
labels_1b = dataset1b['labels']
inputs_1b = dataset1b['input']
bits_1b = dataset1b['bits']
beams_1b = dataset1b['beams']

# Acceder a las variables dentro del archivo (2bit)
labels_2b = dataset2b['labels']
inputs_2b = dataset2b['input']
bits_2b = dataset2b['bits']
beams_2b = dataset2b['beams']

Filtrar datos y procesar tableros

In [65]:
# Filtrar datos de 1 bit
print("Filtrando datos de 1 bit...")
datos1b_filtrados_input, datos1b_filtrados_labels = filtrar_datos_en_rango(inputs_1b, labels_1b)

# Filtrar datos de 2 bits
print("Filtrando datos de 2 bits...")
datos2b_filtrados_input, datos2b_filtrados_labels = filtrar_datos_en_rango(inputs_2b, labels_2b)

print("Datos filtrados listos.")

# Procesar los tableros
print("Procesando tableros...")

# 1 bit
tableros_procesados_1b = procesar_tableros(datos1b_filtrados_labels, 1)

# 2 bit
tableros_procesados_2b = procesar_tableros(datos2b_filtrados_labels, 2)

# Verificar dimensiones antes de transponer
print("Dimensiones antes de reshaping:")
print("tableros_procesados_1b.shape:", tableros_procesados_1b.shape)
print("tableros_procesados_2b.shape:", tableros_procesados_2b.shape)

# Reshape tableros
rs_tableros_procesados_1b = tableros_procesados_1b.reshape(-1, tableros_procesados_1b.shape[1] * tableros_procesados_1b.shape[2])
rs_tableros_procesados_2b = tableros_procesados_2b.reshape(-1, tableros_procesados_2b.shape[1] * tableros_procesados_2b.shape[2])

print("Dimensiones después de reshaping:")
print("rs_tableros_procesados_1b.shape:", rs_tableros_procesados_1b.shape)
print("rs_tableros_procesados_2b.shape:", rs_tableros_procesados_2b.shape)

# Ajustar las dimensiones de datos1b_filtrados_labels
reshaped_labels_1b = datos1b_filtrados_labels.reshape(-1, datos1b_filtrados_labels.shape[0] * datos1b_filtrados_labels.shape[1])
reshaped_labels_2b = datos2b_filtrados_labels.reshape(-1, datos2b_filtrados_labels.shape[0] * datos2b_filtrados_labels.shape[1])

# Comprobar las dimensiones después del reshaping
print("Dimensiones de reshaped_labels_1b:", reshaped_labels_1b.shape)
print("Dimensiones de reshaped_labels_2b:", reshaped_labels_2b.shape)

# Comprobar las dimensiones de datos1b
print("Dimensiones de datos1b_filtrados_labels:", datos1b_filtrados_labels.shape)
print("Dimensiones de datos2b_filtrados_labels:", datos2b_filtrados_labels.shape)

# Comprobar las dimensiones de datos1b
print("Dimensiones de datos1b_filtrados_inputs:", datos1b_filtrados_input.shape)
print("Dimensiones de datos2b_filtrados_inputs:", datos2b_filtrados_input.shape)



Filtrando datos de 1 bit...
Filtrando datos de 2 bits...
Datos filtrados listos.
Procesando tableros...
Dimensiones antes de reshaping:
tableros_procesados_1b.shape: (9180, 10, 10)
tableros_procesados_2b.shape: (9180, 10, 10)
Dimensiones después de reshaping:
rs_tableros_procesados_1b.shape: (9180, 100)
rs_tableros_procesados_2b.shape: (9180, 100)
Dimensiones de reshaped_labels_1b: (9180, 100)
Dimensiones de reshaped_labels_2b: (9180, 100)
Dimensiones de datos1b_filtrados_labels: (10, 10, 9180)
Dimensiones de datos2b_filtrados_labels: (10, 10, 9180)
Dimensiones de datos1b_filtrados_inputs: (9180, 2)
Dimensiones de datos2b_filtrados_inputs: (9180, 2)


Pruebas

In [66]:
# Inicializar matrices para almacenar métricas
predicciones_100_RBFs = np.zeros((10, 10, rs_tableros_procesados_1b.shape[0]))
mse_metrics = np.zeros((10, 10))
mae_metrics = np.zeros((10, 10))
r2_metrics = np.zeros((10, 10))

print(f"rs_tableros_procesados_1b:  ", rs_tableros_procesados_1b.shape)
print(f"datos1b_filtrados_input:  ", datos1b_filtrados_input.shape)

# Definir valores de alpha y sigma para las pruebas
#lambda_regs = [0.01, 0.1, 1.0]
lambda_regs = [0.01, 1]

# Definir valores de alpha, sigma y lambda_reg para las pruebas
alphas_up = []
sigmas_up = []
lambda_up = []

# Realizar pruebas variando num_centers y sigma
for i in range(10):
    for j in range(10):
        y_labels = rs_tableros_procesados_1b[:, i * 10 + j].reshape(-1, 1)
        print(f"Y_labels: ", y_labels.shape)
        X_train1, X_test1, y_train1, y_test1 = train_test_split(
            datos1b_filtrados_input, y_labels, test_size=0.4, random_state=42
        )
        print(f"Datos divididos para la celda ({i},{j}).")

        nc1 = len(X_train1) // 10   # LINEAS NUEVAS 27_6
        #nc1 = 2000 # 2000
        print(f"nc1: ", nc1)
        alphas = np.linspace(nc1 / len(X_train1) / 2, nc1 / len(X_train1) * 2, 10)  #LINEAS NUEVAS 27_6
        print(f"alphas: ", alphas)

        dc = np.linalg.norm(X_train1[:, None] - X_train1[None, :], axis=-1)
        sg = np.median(dc)
        #sg = 7.5 # 15
        print(f"sg: ", sg)
        sigmas = np.linspace(sg / 4, sg * 4, 10)
        print(f"sigmas: ", sigmas)

        # Inicializar matriz para almacenar resultados
        results = np.zeros((len(alphas), len(sigmas), len(lambda_regs)))

        for alpha_index, alpha in enumerate(alphas):
            num_centers = int(np.ceil(len(X_train1) * alpha))
            for sigma_index, sigma in enumerate(sigmas):
                for lambda_reg_index, lambda_reg in enumerate(lambda_regs):
                    #print(f"Probando con num_centers = {num_centers} y sigma = {sigma}...")
                    rbf_net = RBFNetwork(num_centers=num_centers, sigma=sigma, lambda_reg=lambda_reg)
                    rbf_net.fit(X_train1, y_train1)
                    predictions_test = rbf_net.predict(X_test1)
                    mse_test = mean_squared_error(y_test1, predictions_test)
                    results[alpha_index, sigma_index, lambda_reg_index] = mse_test
                    #print(f"MSE en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {mse_test}")

        # Almacenar los resultados de la mejor configuración en las matrices de métricas
        best_alpha_index, best_sigma_index, best_lambda_reg_index = np.unravel_index(np.argmin(results), results.shape)
        best_num_centers = int(np.ceil(len(X_train1) * alphas[best_alpha_index]))
        best_sigma = sigmas[best_sigma_index]
        best_lambda_reg = lambda_regs[best_lambda_reg_index]
        print(f"Best alpha: ", best_num_centers)
        print(f"Best sigma: ", best_sigma)
        print(f"Best lamda_reg: ", best_lambda_reg)
        # Añadir
        alphas_up.append(best_num_centers)
        sigmas_up.append(best_sigma)
        lambda_up.append(best_lambda_reg)


        # Entrenar la RBF con los mejores hiperparámetros
        rbf_net = RBFNetwork(num_centers=best_num_centers, sigma=best_sigma, lambda_reg=best_lambda_reg)
        rbf_net.fit(X_train1, y_train1)
        predictions_all = rbf_net.predict(X_test1)

        # Almacenar las predicciones y métricas
        predicciones_100_RBFs[i, j, :len(predictions_all)] = predictions_all.flatten()
        mse_metrics[i, j] = mean_squared_error(y_test1, predictions_all)
        mae_metrics[i, j] = mean_absolute_error(y_test1, predictions_all)
        r2_metrics[i, j] = r2_score(y_test1, predictions_all)




rs_tableros_procesados_1b:   (9180, 100)
datos1b_filtrados_input:   (9180, 2)
Y_labels:  (9180, 1)
Datos divididos para la celda (0,0).
nc1:  550
alphas:  [0.04992738 0.06656984 0.0832123  0.09985476 0.11649722 0.13313968
 0.14978214 0.16642459 0.18306705 0.19970951]
sg:  56.85068161420758
sigmas:  [ 14.2126704   37.90045441  61.58823842  85.27602242 108.96380643
 132.65159043 156.33937444 180.02715844 203.71494245 227.40272646]
Best alpha:  1009
Best sigma:  14.212670403551895
Best lamda_reg:  0.01
Y_labels:  (9180, 1)
Datos divididos para la celda (0,1).
nc1:  550
alphas:  [0.04992738 0.06656984 0.0832123  0.09985476 0.11649722 0.13313968
 0.14978214 0.16642459 0.18306705 0.19970951]
sg:  56.85068161420758
sigmas:  [ 14.2126704   37.90045441  61.58823842  85.27602242 108.96380643
 132.65159043 156.33937444 180.02715844 203.71494245 227.40272646]
Best alpha:  825
Best sigma:  14.212670403551895
Best lamda_reg:  1
Y_labels:  (9180, 1)
Datos divididos para la celda (0,2).
nc1:  550
alph

Guardar datos

In [67]:
# Display results
print("Resultados de las métricas para la celda (0,0):")
print(f"Celda (0,0): MSE={mse_metrics[0, 0]}, MAE={mae_metrics[0, 0]}, R2={r2_metrics[0, 0]}")

# Display results
print("Resultados de las métricas para la celda (0,1):")
print(f"Celda (0,1): MSE={mse_metrics[0, 1]}, MAE={mae_metrics[0, 1]}, R2={r2_metrics[0, 1]}")

# Display results
print("Resultados de las métricas para la celda (0,2):")
print(f"Celda (0,2): MSE={mse_metrics[0, 2]}, MAE={mae_metrics[0, 2]}, R2={r2_metrics[0, 2]}")

print(f"mse_metrics: ", mse_metrics)
print(f"mae_metrics: ", mae_metrics)
print(f"r2_metrics: ", r2_metrics)



Resultados de las métricas para la celda (0,0):
Celda (0,0): MSE=0.015842440878223063, MAE=0.03727049829405457, R2=0.06049302516832067
Resultados de las métricas para la celda (0,1):
Celda (0,1): MSE=0.12554326614453754, MAE=0.2577257662847004, R2=0.24100146767110542
Resultados de las métricas para la celda (0,2):
Celda (0,2): MSE=0.12410010386970576, MAE=0.2572094020574214, R2=0.4552458714263846
mse_metrics:  [[0.01584244 0.12554327 0.1241001  0.14435834 0.15745871 0.17268277
  0.19017855 0.19986309 0.20797394 0.21571842]
 [0.12631897 0.13593855 0.13283637 0.14265393 0.16236532 0.18200111
  0.19164283 0.20088458 0.20920352 0.21861837]
 [0.125518   0.13117791 0.14114012 0.14218038 0.16761039 0.18758969
  0.19911244 0.20269494 0.20850084 0.21437937]
 [0.14194924 0.13704201 0.14432424 0.16747592 0.17650201 0.18988584
  0.19887668 0.20549146 0.21151834 0.22197559]
 [0.15839279 0.16224813 0.16132427 0.17866087 0.19158474 0.19618406
  0.2052697  0.21195319 0.21964615 0.22218122]
 [0.1775946

Guardar datos (all)

In [69]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_metric_vs_hyperparam(metric_values, hyperparam_values, hyperparam_name, metric_name):
    plt.figure()
    plt.scatter(hyperparam_values, metric_values)
    plt.xlabel(hyperparam_name)
    plt.ylabel(metric_name)
    plt.title(f'{metric_name} vs {hyperparam_name}')
    plt.grid(True)
    return plt.gcf()


# Crear directorio 'img' si no existe
if not os.path.exists('img'):
    os.makedirs('img')

# Guardar resultados en un archivo Excel
workbook = xlsxwriter.Workbook('resultados_rbf3.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write(0, 0, 'Celda')
worksheet.write(0, 1, 'MSE')
worksheet.write(0, 2, 'MAE')
worksheet.write(0, 3, 'R2')

for i in range(10):
    for j in range(10):
        worksheet.write(i * 10 + j + 1, 0, f'({i},{j})')
        worksheet.write(i * 10 + j + 1, 1, mse_metrics[i, j])
        worksheet.write(i * 10 + j + 1, 2, mae_metrics[i, j])
        worksheet.write(i * 10 + j + 1, 3, r2_metrics[i, j])

workbook.close()

# Graficar superficies 3D para MSE, MAE y R2
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(range(10), range(10))
ax.plot_surface(X, Y, mse_metrics, cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('MSE')
plt.title('Superficie de MSE por celda')
plt.savefig('img/mse_surface.png')
plt.close(fig)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, mae_metrics, cmap='plasma')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('MAE')
plt.title('Superficie de MAE por celda')
plt.savefig('img/mae_surface.png')
plt.close(fig)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, r2_metrics, cmap='inferno')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('R2')
plt.title('Superficie de R2 por celda')
plt.savefig('img/r2_surface.png')
plt.close(fig)

# Mostrar resultados
print("Resultados de las métricas:")
for i in range(10):
    for j in range(10):
        print(f"Celda ({i},{j}): MSE={mse_metrics[i, j]}, MAE={mae_metrics[i, j]}, R2={r2_metrics[i, j]}")

# Inicializar listas para almacenar los resultados de las métricas por hiperparámetro
alpha_mse = []
alpha_mae = []
alpha_r2 = []
sigma_mse = []
sigma_mae = []
sigma_r2 = []
lambda_mse = []
lambda_mae = []
lambda_r2 = []

sigmas_up = [float(value) for value in sigmas_up]

print(f"Alphas_up: ", alphas_up)
print(f"Sigmas_up: ", sigmas_up)
print(f"Lambda_up: ", lambda_up)

for i in range(10):
    for j in range(10):
        if mse_metrics[i, j] != 0:
            alpha_mse.append(mse_metrics[i, j])
            alpha_mae.append(mae_metrics[i, j])
            alpha_r2.append(r2_metrics[i, j])
            sigma_mse.append(mse_metrics[i, j])
            sigma_mae.append(mae_metrics[i, j])
            sigma_r2.append(r2_metrics[i, j])
            lambda_mse.append(mse_metrics[i, j])
            lambda_mae.append(mae_metrics[i, j])
            lambda_r2.append(r2_metrics[i, j])

alpha_mse = [float(value) for value in alpha_mse]
alpha_mae = [float(value) for value in alpha_mae]
alpha_r2 = [float(value) for value in alpha_r2]

sigma_mse = [float(value) for value in sigma_mse]
sigma_mae = [float(value) for value in sigma_mae]
sigma_r2 = [float(value) for value in sigma_r2]

lambda_mse = [float(value) for value in lambda_mse]
lambda_mae = [float(value) for value in lambda_mae]
lambda_r2 = [float(value) for value in lambda_r2]

print(f"Alphas_mse ", alpha_mse)
print(f"Alphas_mae ", alpha_mae)
print(f"Alphas_r2 ", alpha_r2)

print(f"Sigmas_mse: ", sigma_mse)
print(f"Sigmas_mae: ", sigma_mae)
print(f"Sigmas_r2: ", sigma_r2)

print(f"Lambda_mse: ", lambda_mse)
print(f"Lambda_mae: ", lambda_mae)
print(f"Lambda_r2: ", lambda_r2)

# Graficar optimización de hiperparámetros
with PdfPages('hyperparam_optimization_results.pdf') as pdf:
    pdf.savefig(plot_metric_vs_hyperparam(alpha_mse, alphas_up, 'Alpha', 'MSE'))
    pdf.savefig(plot_metric_vs_hyperparam(alpha_mae, alphas_up, 'Alpha', 'MAE'))
    pdf.savefig(plot_metric_vs_hyperparam(alpha_r2, alphas_up, 'Alpha', 'R2'))
    
    pdf.savefig(plot_metric_vs_hyperparam(sigma_mse, sigmas_up, 'Sigma', 'MSE'))
    pdf.savefig(plot_metric_vs_hyperparam(sigma_mae, sigmas_up, 'Sigma', 'MAE'))
    pdf.savefig(plot_metric_vs_hyperparam(sigma_r2, sigmas_up, 'Sigma', 'R2'))
    
    pdf.savefig(plot_metric_vs_hyperparam(lambda_mse, lambda_up, 'Lambda', 'MSE'))
    pdf.savefig(plot_metric_vs_hyperparam(lambda_mae, lambda_up, 'Lambda', 'MAE'))
    pdf.savefig(plot_metric_vs_hyperparam(lambda_r2, lambda_up, 'Lambda', 'R2'))

print("Gráficos guardados en hyperparam_optimization_results.pdf")

# Gráfico Real vs Predicted para una celda
fig, ax = plt.subplots()
real_values = y_test1.flatten()
predicted_values = predictions_all.flatten()
ax.scatter(real_values, predicted_values, edgecolors=(0, 0, 0))
ax.plot([real_values.min(), real_values.max()], [real_values.min(), real_values.max()], 'k--', lw=4)
ax.set_xlabel('Valores Reales')
ax.set_ylabel('Valores Predichos')
plt.title('Valores Reales vs Valores Predichos')
plt.savefig('img/real_vs_predicted.png')
plt.close(fig)

# Gráficos de valores reales vs predichos para todas las celdas
for i in range(10):
    for j in range(10):
        real_values = (y_test1 > 0.5).astype(int).flatten()
        predicted_values = (predicciones_100_RBFs[i, j, :len(y_test1)] > 0.5).astype(int).flatten()

        fig, ax = plt.subplots()
        ax.scatter(real_values, predicted_values, edgecolors=(0, 0, 0))
        ax.plot([real_values.min(), real_values.max()], [real_values.min(), real_values.max()], 'k--', lw=4)
        ax.set_xlabel('Valores Reales')
        ax.set_ylabel('Valores Predichos')
        plt.title(f'Valores Reales vs Valores Predichos para la celda ({i},{j})')
        plt.savefig(f'img/real_vs_predicted_{i}_{j}.png')
        plt.close(fig)

        # Crear matriz de confusión para cada celda
        cm = confusion_matrix(real_values, predicted_values)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.title(f'Matriz de Confusión para la celda ({i},{j})')
        plt.savefig(f'img/confusion_matrix_{i}_{j}.png')
        plt.close()



Resultados de las métricas:
Celda (0,0): MSE=0.015842440878223063, MAE=0.03727049829405457, R2=0.06049302516832067
Celda (0,1): MSE=0.12554326614453754, MAE=0.2577257662847004, R2=0.24100146767110542
Celda (0,2): MSE=0.12410010386970576, MAE=0.2572094020574214, R2=0.4552458714263846
Celda (0,3): MSE=0.14435834433210487, MAE=0.29136034259014243, R2=0.3885582767447392
Celda (0,4): MSE=0.15745871432472738, MAE=0.3198604336239407, R2=0.3239578075789732
Celda (0,5): MSE=0.17268276604925686, MAE=0.35686813249236554, R2=0.29121787865516
Celda (0,6): MSE=0.1901785461599851, MAE=0.3845860660892037, R2=0.2273707762826983
Celda (0,7): MSE=0.19986309220223195, MAE=0.40190942317781997, R2=0.185811844268841
Celda (0,8): MSE=0.20797393851800713, MAE=0.41743635733220374, R2=0.14666749746489982
Celda (0,9): MSE=0.21571841589186544, MAE=0.43399155502718884, R2=0.12262989230667742
Celda (1,0): MSE=0.1263189702731015, MAE=0.25981239740210527, R2=0.20611767782151358
Celda (1,1): MSE=0.13593854694907348, MA

C:\Users\ismae\AppData\Local\Temp\ipykernel_14664\2219620117.py:134: MatplotlibDeprecationWarning: Keeping empty pdf files is deprecated since 3.8 and support will be removed in 3.10.
  with PdfPages('hyperparam_optimization_results.pdf') as pdf:


TypeError: plot_metric_vs_hyperparam() missing 1 required positional argument: 'file_name'